In [39]:
import os
import sys
import requests
from cachetools import cached, TTLCache
import time
import pandas as pd
import tqdm

g_token=''

@cached(cache=TTLCache(maxsize=1, ttl=36000))
def get_token():
    # step 1:  using username='30007', password='2A4nFXhP' 
    ##         on https://mid.meddra.org/account/register for apikey
    # step 2:  apply for grant
    cmd = '''
    curl \
        --request POST \
        --data "grant_type=password"  \
        --data "username={username}"  --data "password={apikey}" \
        --data "scope=meddraapi" \
        --data "client_id=mspclient" \
        "https://mid.meddra.org/connect/token"
    '''.format(username='30007', apikey='4Lj68iPSy3jjti7hK2hM')
    os.system(cmd)
    # ref: https://cloud.tencent.com/developer/article/1908468
    return auth['access_token']
    # {"access_token":"6acb65aaaa2eee65530de8e2521606b10051a5b8a5451bdf1227eb704bda080d","expires_in":36000,"token_type":"Bearer"}

def get_token2():
    return "026fbb4fcd254ffdfe0ea227ac4be437dff38d5896cc9825e78a5166d865ff2e"  #8-12 19:00

def get_soc(row):
    ## (1) search pt name for pt-code
    # considering sleep 3s
    time.sleep(1)
    
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer {}'.format(get_token2()),
        'Content-Type': 'text/json',
    }
    if type(row) == type('a'):
        pt_name = row
    else:
        pt_name = row['pt']   # "heart valve operation"

    data = '{"bview":"SOC","rsview":"release","language":"English","stype":1,"addlangs":[],"filters":[],"version":24,"searchterms":[{"searchtype":0,"searchterm":"' \
           + pt_name + '","searchlogic":0}],"kana":false,"idiacritical":true,"synonym":false,"contains":false,"soc":false,"hlgt":false,"hlt":false,"pt":true,"llt":false,"smq":false,"skip":0,"take":0,"separator":2}'
    
    try:
        response = requests.post('https://gxpmapiprod.meddra.org/api/search', headers=headers, data=data)
        #print(response.json())
        ##(2) detail: search soc name & code using pt_code
        pt_code = response.json()[0]['pcode']
    
        data = '{"bview":"SOC","rsview":"release","code":' + str(pt_code) + \
               ',"pcode":0,"syncode":0,"lltcode":0,"ptcode":0,"hltcode":0,"hlgtcode":0,"soccode":0,"smqcode":0,"type":"PT","addlangs":[],"rtype":"P","lang":"English","ver":24,"kana":false,"separator":2}'
    
        response = requests.post('https://gxpmapiprod.meddra.org/api/detail', headers=headers, data=data)
        soc_name = ''
        soc_code = -1
    
        for item in response.json()['mds']:
            #print(item)
            if item['primarysocfg'] == 'Y':
                soc_name = item['socname']
                soc_code = item['soccode']
                           
        ## finally output results using dataframes
        #return {'pt': pt_name, 'pt_code': pt_code, 'soc_name': soc_name, 'soc_code': soc_code}
        return pd.Series([pt_name, pt_code, soc_name, soc_code], index=['pt_name', 'pt_code', 'soc_name', 'soc_code'])

    except Exception as e:
        # 查询不成功赋予默认值
        return pd.Series([pt_name, '-', '-', '-'], index=['pt_name', 'pt_code', 'soc_name', 'soc_code'])
    finally:
        pass


In [40]:
get_soc('benign pancreatic hyperenzymaemia')

pt_name     benign pancreatic hyperenzymaemia
pt_code                                     -
soc_name                                    -
soc_code                                    -
dtype: object

In [16]:
#get_token()
conn.close()

NameError: name 'conn' is not defined

In [ ]:
conn = pymysql.connect(
        host='localhost',
        port=int(3306),
        user="root",
        passwd='0921051011Liu',
        db="faers",
        charset='utf8mb4')

# 去除异常字符
sql = '''   
   SELECT 
        replace(coalesce(t2.pt,'not_found_pt'),'\\r', '') as pt,
        COUNT(distinct t1.CASEID) AS caseid_cnt,
        COUNT(distinct t1.PRIMARYID) AS primaryid_cnt
    FROM faers.ribo_drug_usage t1
    left join faers.faers_reac t2
    on t1.primaryid = t2.primaryid and lower(t1.role_cod) = 'ps'
    group by 1
    order by 1;
'''
df_pt_stat = pd.read_sql_query(sql, conn)

conn.close()

In [18]:
df_pt_stat.describe()

,caseid_cnt,primaryid_cnt
count,3223.000000,3223.000000
mean,21.782811,33.657152
std,94.176502,136.683128
min,1.000000,1.000000
25%,1.000000,2.000000
50%,3.000000,4.000000
75%,9.000000,15.000000
max,2030.000000,2733.000000


In [42]:
get_soc('buttock claudication')

pt_name     buttock claudication
pt_code                        -
soc_name                       -
soc_code                       -
dtype: object

In [26]:
import tqdm
from tqdm import tqdm
tqdm.pandas(desc='apply')

df_pt_stat=pd.read_csv('./ribo_pt.csv')
pt_soc_mapping = df_pt_stat.progress_apply(get_soc, axis=1)
pt_soc_mapping.to_csv('./pt_soc_mapping.csv', index=False)
#pt_soc_mapping = df_pt_stat.apply(get_soc, axis=1)



apply:   9%|▊         | 281/3223 [19:48<3:06:09,  3.80s/it]

error for: benign pancreatic hyperenzymaemia


apply:   9%|▉         | 283/3223 [19:55<2:51:54,  3.51s/it]

error for: bile duct obstruction


apply:  13%|█▎        | 418/3223 [29:33<2:46:14,  3.56s/it]

...

apply:  89%|████████▉ | 2867/3223 [3:25:19<19:31,  3.29s/it]

error for: superficial vein thrombosis


apply:  90%|█████████ | 2914/3223 [3:29:02<23:17,  4.52s/it]

error for: thalamic stroke


apply: 100%|██████████| 3223/3223 [3:51:47<00:00,  4.31s/it]


In [30]:
pt_soc_mapping[pt_soc_mapping['pt_name'] == 'benign pancreatic hyperenzymaemia']

,pt_name,pt_code,soc_name,soc_code


In [33]:
len(df_pt_stat)
df_pt_stat[df_pt_stat['pt'] == 'benign pancreatic hyperenzymaemia']


,pt,caseid_cnt,primaryid_cnt
279,benign pancreatic hyperenzymaemia,1,1


In [ ]:
error for: anal inflammation
error for: anaphylactic shock
error for: aneurysm
error for: anger
error for: aspartate aminotransferase abnormal
error for: aspartate aminotransferase increased
error for: benign pancreatic hyperenzymaemia
error for: bile duct obstruction
error for: biliary tract disorder
error for: bilirubin conjugated abnormal
error for: bilirubin conjugated increased
error for: biopsy breast
error for: blood viscosity decreased
error for: body mass index abnormal
error for: body mass index decreased
error for: body mass index increased
error for: bone demineralisation
error for: bone pain
error for: bone scan abnormal
error for: brain fog
error for: brca1 gene mutation
error for: brca2 gene mutation
error for: breast calcifications
error for: breast cancer
error for: breast cancer female
error for: buttock claudication
error for: calcification metastatic
error for: calcinosis
error for: calciphylaxis
error for: calcium deficiency
error for: cancer fatigue
error for: cerebral ischaemia
error for: cerebral microangiopathy
error for: cholecystitis acute
error for: cholecystitis chronic
error for: cholelithiasis
error for: congenital cystic kidney disease
error for: congestive cardiomyopathy
error for: conjunctival haemorrhage
error for: conjunctivitis
error for: connective tissue disorder
error for: contraindicated product administered
error for: contraindicated product prescribed
error for: craniocerebral injury
error for: creatinine renal clearance decreased
error for: creatinine renal clearance increased
error for: crepitations
error for: creutzfeldt-jakob disease
error for: device related sepsis
error for: dilated cardiomyopathy
error for: drug dose omission
error for: drug effect incomplete